
# 📘 FB3PFB — NHANES (Intro notebook)

**Goal:** a very simple, step-by-step analysis in Google Colab, with no installations or Drive.  
What we’ll do:
- Load minimal libraries.
- Download **DEMO_J**, **HDL_J**, **TRIGLY_J** directly from the CDC site.
- Recode `riagendr` → `sex` (Male/Female).
- Convert HDL, triglycerides from **mg/dL** → **mmol/L**.
- Show mean/SD/median/IQR for **age** by **sex**.
- Draw a **density plot** of HDL by sex.
- Compute **difference in HDL** (Female − Male) and a **t-test**.
- Make a **scatter plot** of HDL vs Age.
- Fit a tiny **OLS regression**: HDL ~ sex + age.

> This notebook is intentionally simple (no custom functions unless necessary).


## 1) Load libraries

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
from pathlib import Path

# For downloading files from the web (CDC)
import requests

# Plotting
import matplotlib.pyplot as plt

# Statistics
from scipy import stats
import statsmodels.formula.api as smf


## 2) Download DEMO, HDL, TRIGLY from CDC

We save to `/content/data/raw` (Colab's temporary filesystem).  
We use a small dictionary (key = filename, value = URL) and loop over it to download each file.


In [ ]:
# Where to store data in Colab
DATA_DIR = Path("/content/data/raw")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# CDC links (2017–2018 cycle "_J"). Adjust if you need a different cycle.
FILES = {
    "DEMO_J.xpt":   "https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.XPT",
    "HDL_J.xpt":    "https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/HDL_J.XPT",
    "TRIGLY_J.xpt": "https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/TRIGLY_J.XPT",
}

# Simple download loop:
for fname, url in FILES.items():
    dest = DATA_DIR / fname
    if not dest.exists():
        print(f"Downloading {fname} ...")
        r = requests.get(url, timeout=60)
        r.raise_for_status()
        with open(dest, "wb") as f:
            f.write(r.content)
    else:
        print(f"Already have {fname}")

print("\nFiles in", DATA_DIR)
for p in sorted(DATA_DIR.iterdir()):
    print(" -", p.name)

## 3) Load the three files into DataFrames

We read **XPT** files with `pandas.read_sas(..., format="xport")` and lower-case the columns.


In [ ]:
# Straight loading (no helpers)
p_demo   = DATA_DIR / "DEMO_J.xpt"
p_hdl    = DATA_DIR / "HDL_J.xpt"
p_trigly = DATA_DIR / "TRIGLY_J.xpt"

df_demo = pd.read_sas(p_demo, format="xport", encoding="utf-8")
df_hdl  = pd.read_sas(p_hdl,  format="xport", encoding="utf-8")
df_trig = pd.read_sas(p_trigly, format="xport", encoding="utf-8")

# Lowercase all columns
df_demo.columns = [c.lower() for c in df_demo.columns]
df_hdl.columns  = [c.lower() for c in df_hdl.columns]
df_trig.columns = [c.lower() for c in df_trig.columns]

print("Shapes:", df_demo.shape, df_hdl.shape, df_trig.shape)
df_demo.head()

## 4) Minimal merges (left-join labs onto demographics by `seqn`)

NHANES uses `SEQN` (lowercased here to `seqn`) as the participant ID.


In [ ]:
# Keep essentials from DEMO to stay tidy
demo_keep = ["seqn", "riagendr", "ridageyr"]
d = df_demo[demo_keep].copy()

# Left-join HDL and triglycerides (drop_duplicates just in case)
d = d.merge(df_hdl.drop_duplicates(subset=["seqn"]), on="seqn", how="left")
d = d.merge(df_trig.drop_duplicates(subset=["seqn"]), on="seqn", how="left")

# Peek at likely HDL/TG column names (NHANES names vary a bit by file)
print([c for c in d.columns if "hdl" in c.lower()][:10])
print([c for c in d.columns if "trig" in c.lower()][:10])

## 5) Recode sex; convert HDL & triglycerides to SI units

- `riagendr`: **1 → Male**, **2 → Female**  
- HDL: **mg/dL → mmol/L** multiply by **0.02586**  
- Triglycerides: **mg/dL → mmol/L** multiply by **0.01129**


In [ ]:
# Recode sex
d["riagendr"] = pd.to_numeric(d["riagendr"], errors="coerce")
d["sex"] = d["riagendr"].map({1: "Male", 2: "Female"})

# Identify HDL & triglyceride columns
hdl_col_candidates  = [c for c in d.columns if "hdl"  in c.lower()]
trig_col_candidates = [c for c in d.columns if "trig" in c.lower()]

print("HDL candidates:", hdl_col_candidates[:5])
print("Triglyceride candidates:", trig_col_candidates[:5])

# Pick specific columns (edit if needed)
hdl_mgdl_col  = "lbdhdd" if "lbdhdd" in d.columns else hdl_col_candidates[0]
trig_mgdl_col = "lbxtr"  if "lbxtr"  in d.columns else trig_col_candidates[0]

# Ensure numeric
d[hdl_mgdl_col]  = pd.to_numeric(d[hdl_mgdl_col],  errors="coerce")
d[trig_mgdl_col] = pd.to_numeric(d[trig_mgdl_col], errors="coerce")
d["ridageyr"]    = pd.to_numeric(d["ridageyr"],    errors="coerce")

# Create SI-unit columns
d["hdl_mmol_l"]  = d[hdl_mgdl_col]  * 0.02586
d["tg_mmol_l"]   = d[trig_mgdl_col] * 0.01129

d[["sex", "ridageyr", hdl_mgdl_col, "hdl_mmol_l", trig_mgdl_col, "tg_mmol_l"]].head()

## 6) Show mean/SD/median/IQR for **age**, by **sex**

We loop over the two sex groups and print simple summaries.


In [ ]:
print("=== Age summary by sex ===")
for grp in ["Male", "Female"]:
    sub = d.loc[d["sex"] == grp, "ridageyr"].dropna()
    if sub.empty:
        print(f"\n{grp}: no data")
        continue

    n = sub.size
    mean = sub.mean()
    sd   = sub.std(ddof=1)
    median = sub.median()
    q25 = sub.quantile(0.25)
    q75 = sub.quantile(0.75)

    print(f"\n{grp}: n={n}")
    print(f"  Mean ± SD:    {mean:.1f} ± {sd:.1f}")
    print(f"  Median [IQR]: {median:.1f} [{q25:.1f}, {q75:.1f}]")

## 7) Density plot (HDL in mmol/L by sex)

Two kernel density curves, one per sex.


In [ ]:
# Prepare series (drop missing)
hdl_f = d.loc[d["sex"] == "Female", "hdl_mmol_l"].dropna()
hdl_m = d.loc[d["sex"] == "Male",   "hdl_mmol_l"].dropna()

plt.figure()
hdl_f.plot(kind="kde", linewidth=2, label="Female")
hdl_m.plot(kind="kde", linewidth=2, label="Male")
plt.xlabel("HDL (mmol/L)")
plt.ylabel("Density")
plt.title("HDL distribution by sex")
plt.legend()
plt.show()

## 8) Difference in HDL (Female − Male)

Simple unweighted mean difference in mmol/L.


In [ ]:
hdl_mean_f = hdl_f.mean()
hdl_mean_m = hdl_m.mean()
diff = hdl_mean_f - hdl_mean_m

print(f"Mean HDL (mmol/L): Female={hdl_mean_f:.2f}, Male={hdl_mean_m:.2f}")
print(f"Difference (Female − Male): {diff:.2f} mmol/L")

## 9) t-test (Welch’s, safer default)

Welch’s t-test does not assume equal variances.


In [ ]:
t_stat, p_val = stats.ttest_ind(hdl_f, hdl_m, equal_var=False, nan_policy="omit")
print("Welch's t-test on HDL (mmol/L): Female vs Male")
print(f"  t = {t_stat:.3f}, p = {p_val:.3g}")
print(f"  n_female = {hdl_f.size}, n_male = {hdl_m.size}")

## 10) Scatter plot: HDL (mmol/L) vs Age (years)

Points colored by sex.


In [ ]:
plt.figure()
for label, sub in d.dropna(subset=["ridageyr", "hdl_mmol_l"]).groupby("sex", dropna=False):
    plt.scatter(sub["ridageyr"], sub["hdl_mmol_l"], s=10, alpha=0.4, label=str(label))
plt.xlabel("Age (years)")
plt.ylabel("HDL (mmol/L)")
plt.title("HDL vs Age by sex")
plt.legend()
plt.show()

## 11) Regression: HDL (mmol/L) ~ sex + age

We treat `sex` as categorical using `C(sex)` and drop rows with missing values in model variables.


In [ ]:
reg = d[["hdl_mmol_l", "sex", "ridageyr"]].dropna().copy()
model = smf.ols("hdl_mmol_l ~ C(sex) + ridageyr", data=reg).fit()
print(model.summary())